In [ ]:
import re

# Export all layers

Utiliser le plugin `Batch Save Vector layers` pour exporter tous les fichiers **shapefile** dans un dossier en vrac.

Réordonner les fichiers selon une architecture pertinente (données d'un côté, potentiels de l'autre, etc.).

Sauvegarder le fichier `.qgz` dans la même arborescence que les fichiers **shapefile** (par exemple, le fichier `.qgz` dans un dossier et les fichiers `.shp` dans un sous-dossier *layers* du même dossier).

In [ ]:
path_to_folder = """/mnt/c/Users/ville/Dropbox (Villes Vivantes)/Villes Vivantes/affaires en cours/EPERNAY PUCA SOBRIETE FONCIERE/3_OPERATIONNEL/1_SIG/"""
path_to_folder = """/mnt/c/Users/ville/Dropbox (Villes Vivantes)/Villes Vivantes/affaires en cours/ETUDE D'OPTIMISATION DU FONCIER POLE METROPOLITAIN LOIRE ANGERS - 2021/SIG/"""

In [ ]:
qgz_file_name = 'epernay_fonciers_potentiels'
qgz_file_name = 'pmla_fonciers_potentiels'

On extrait d'abord l'archive et plus particulièrement le fichier `.qgs`

In [ ]:
import zipfile, io
archive = zipfile.ZipFile(path_to_folder + f'/{qgz_file_name}.qgz', 'r')
with archive.open(f'{qgz_file_name}.qgs') as f:
    qgs_content = f.read().decode('utf-8')
archive.close()

On détecte ensuite tous les champs qui correspondent à une source de données :

In [ ]:
reg_name_src = re.compile(r' name="([^"]+)" ([^>]+) source="([^"]+)"')

In [ ]:
names_src = reg_name_src.findall(qgs_content)

On affiche ceux qui correspondent déjà à des shapefiles qu'il va falloir **déplacer**

In [ ]:
for g in names_src:
    name = g[0]
    src = g[2]
    if ".shp" in src:
        print(name, src)

On effectue ensuite les remplacements de source :

In [ ]:
reg_name_src = re.compile(r' name="([^"]+)" ([^>]*)source="([^"]+)"')
qgs_content_fixed = reg_name_src.sub(r' name="\1" \2 source="./layers/\1.shp"', qgs_content)
reg_name_src = re.compile(r' source="([^"]+)" ([^>]*)name="([^"]+)"')
qgs_content_fixed = reg_name_src.sub(r' source="./layers/\3.shp" \2 name="\3"', qgs_content_fixed)
reg_data_source = re.compile(r'<datasource>((?:(?!\.csv|<).)+)<\/datasource>(?!layername)(?!datasource)(.+?)<layername>([^<]+)<\/layername>',
                            flags=re.MULTILINE | re.DOTALL)
qgs_content_fixed = reg_data_source.sub(r'<datasource>./layers/\3.shp</datasource>\2<layername>\3</layername>', qgs_content_fixed)
qgs_content_fixed = qgs_content_fixed.replace('providerKey="postgres"', 'providerKey="ogr"')
qgs_content_fixed = qgs_content_fixed.replace('<provider encoding="">postgres</provider>', '<provider encoding="UTF-8">ogr</provider>')
print(qgs_content_fixed)

Pour un export en **.qgs**

In [ ]:
if False:
    with open(path_to_folder + f'/{qgz_file_name}_moved.qgs', "w") as f:
        f.write(qgs_content_fixed)

Pour écraser le fichier initial (une sauvegarde est mise) :

In [ ]:
import shutil

In [ ]:
shutil.copyfile(path_to_folder + f'/{qgz_file_name}.qgz', path_to_folder + f'/{qgz_file_name}_local.qgz')

In [ ]:
archive_write = zipfile.ZipFile(path_to_folder + f'/{qgz_file_name}.qgz', 'w', compression=zipfile.ZIP_DEFLATED)
print(archive_write.writestr(f'{qgz_file_name}.qgs', qgs_content_fixed))
archive_write.close()